<a href="https://colab.research.google.com/github/jangvu/Project_by_me/blob/main/resnet_34.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from functools import partial


In [ ]:
DefaultConv2D = partial(keras.layers.Conv2D, kernel_size=3, strides=1,
                        padding="SAME", use_bias=False)

class ResidualUnit(keras.layers.Layer):
    def __init__(self, filters, strides=1, activation="relu", **kwargs):
        super().__init__(**kwargs)
        self.activation = keras.activations.get(activation)
        self.main_layers = [
            DefaultConv2D(filters, strides=strides),
            keras.layers.BatchNormalization(),
            self.activation,
            DefaultConv2D(filters),
            keras.layers.BatchNormalization()]
        self.skip_layers = []
        if strides > 1:
            self.skip_layers = [
                DefaultConv2D(filters, kernel_size=1, strides=strides),
                keras.layers.BatchNormalization()]

    def call(self, inputs):
        Z = inputs
        for layer in self.main_layers:
            Z = layer(Z)
        skip_Z = inputs
        for layer in self.skip_layers:
            skip_Z = layer(skip_Z)
        return self.activation(Z + skip_Z)


In [ ]:
model = keras.models.Sequential()
model.add(DefaultConv2D(64, kernel_size=7, strides=2,
                        input_shape=[32, 32, 3]))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Activation("relu"))
model.add(keras.layers.MaxPool2D(pool_size=3, strides=2, padding="SAME"))
prev_filters = 64
for filters in [64] * 3 + [128] * 4 + [256] * 6 + [512] * 3:
    strides = 1 if filters == prev_filters else 2
    model.add(ResidualUnit(filters, strides=strides))
    prev_filters = filters
model.add(keras.layers.GlobalAvgPool2D())
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(10, activation="softmax"))


In [ ]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_781 (Conv2D)          (None, 16, 16, 64)        9408      
_________________________________________________________________
batch_normalization_781 (Bat (None, 16, 16, 64)        256       
_________________________________________________________________
activation_3 (Activation)    (None, 16, 16, 64)        0         
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 8, 8, 64)          0         
_________________________________________________________________
residual_unit_281 (ResidualU (None, 8, 8, 64)          74240     
_________________________________________________________________
residual_unit_282 (ResidualU (None, 8, 8, 64)          74240     
_________________________________________________________________
residual_unit_283 (ResidualU (None, 8, 8, 64)         

In [ ]:
(X_train_full,y_train_full),(X_test,y_test) = tf.keras.datasets.cifar10.load_data()


In [ ]:
X_train,X_valid = X_train_full[:5000]/255.0, X_train_full[5000:]/255.0
y_train,y_valid = y_train_full[:5000], y_train_full[5000:]
X_train.shape

(5000, 32, 32, 3)

In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer='Nadam',metrics='accuracy')
#early_stop = keras.callbacks.ModelCheckpoint()

In [ ]:
history = model.fit(X_train,y_train,epochs = 10, validation_data=(X_valid,y_valid))

Epoch 1/10


ValueError: ignored